In [1]:
import os
import warnings

# Suppress tqdm IProgress warning in Jupyter (optional: pip install ipywidgets for widget progress bars)
try:
    from tqdm import TqdmWarning
    warnings.filterwarnings("ignore", category=TqdmWarning)
except ImportError:
    pass

import pandas as pd
from openai import OpenAI

In [2]:
# read case data from csv file
case_data = pd.read_csv('data/dummy_data_clean.csv')

In [3]:
# convert case_data.case_text_lower to Kura Conversation object

from kura.types import Message, Conversation
from datetime import datetime
from rich import print


def process_case_row(row) -> Conversation:
    """Build a Kura Conversation from a case_data row (case_id, case_text_lower)."""
    case_id = row["case_id"]
    case_text = row["case_text_lower"] if pd.notna(row["case_text_lower"]) else ""
    return Conversation(
        chat_id=str(case_id),
        created_at=datetime.now(),
        messages=[
            Message(
                created_at=datetime.now(), # replace with case_data.created_at
                # product = case_data.product
                role="user",
                content=case_text,
            )
        ],
        metadata={"case_id": case_id},
    )


# Build conversations from case_data and show the first one
conversations = [process_case_row(case_data.iloc[i]) for i in range(len(case_data))]
print(conversations[0])


Conversation(
    chat_id='65047504',
    created_at=datetime.datetime(2026, 2, 18, 15, 56, 49, 536096),
    messages=[
        Message(
            created_at=datetime.datetime(2026, 2, 18, 15, 56, 49, 536102),
            role='user',
            content='product: time & labor | case type: support requested - time & labor | case detail: time & 
labor | subject: holiday | description: helped the client with the holiday list'
        )
    ],
    metadata={'case_id': 65047504.0}
)

In [4]:
from kura.checkpoints import JSONLCheckpointManager
from kura.summarisation import summarise_conversations, SummaryModel
from kura.cluster import generate_base_clusters_from_conversation_summaries, ClusterDescriptionModel
from kura.meta_cluster import reduce_clusters_from_base_clusters, MetaClusterModel
from kura.dimensionality import reduce_dimensionality_from_clusters, HDBUMAP
from kura.visualization import visualise_pipeline_results



In [6]:
# Model setup from config: switch between OpenAI / Bedrock (summary) and OpenAI / local (embeddings)
import config
from config import (
    SUMMARIZATION_BACKEND,
    SUMMARIZATION_MODEL,
    EMBEDDING_BACKEND,
    EMBEDDING_MODEL,
    get_checkpoint_dir,
)
from kura.embedding import OpenAIEmbeddingModel, SentenceTransformerEmbeddingModel

CHECKPOINT_DIRECTORY = str(get_checkpoint_dir())


def get_summary_model():
    """Build SummaryModel from config. Use OpenAI or AWS Bedrock via Instructor provider string."""
    # Instructor format: "provider/model-id"
    # OpenAI:   "openai/gpt-4o-mini", "openai/gpt-4o"
    # Bedrock:  "bedrock/anthropic.claude-3-5-sonnet-20241022-v2:0" (set AWS credentials / region)
    if SUMMARIZATION_BACKEND == "bedrock":
        model_id = f"bedrock/{SUMMARIZATION_MODEL}"
    else:
        model_id = f"openai/{SUMMARIZATION_MODEL}"
    return SummaryModel(model=model_id, max_concurrent_requests=10)


def get_embedding_model():
    """Build embedding model from config. Use OpenAI API or local SentenceTransformer (e.g. all-mpnet-base-v2)."""
    if EMBEDDING_BACKEND == "local":
        # Local: sentence-transformers (pip install sentence-transformers)
        # Examples: "all-mpnet-base-v2", "all-MiniLM-L6-v2"
        return SentenceTransformerEmbeddingModel(
            model_name=EMBEDDING_MODEL,
            model_batch_size=128,
            device="cpu",  # or "cuda" if GPU
        )
    else:
        # OpenAI: "text-embedding-3-small", "text-embedding-3-large"
        return OpenAIEmbeddingModel(
            model_name=EMBEDDING_MODEL,
            model_batch_size=50,
            n_concurrent_jobs=5,
        )


# Optional: print active config
print(f"Summary: {SUMMARIZATION_BACKEND} / {SUMMARIZATION_MODEL}")
print(f"Embedding: {EMBEDDING_BACKEND} / {EMBEDDING_MODEL}")
print(f"Checkpoints: {CHECKPOINT_DIRECTORY}")

Summary: openai / gpt-4o-mini

Embedding: local / all-mpnet-base-v2

Checkpoints: /Users/Dev/Documents/coding/case-clustering-kura/checkpoints

In [7]:
# from kura.summarisation import SummaryModel
# from rich import print as rprint

# summaries = await SummaryModel().summarise(conversations[:2])
# for summary in summaries:
#     rprint(summary)

In [8]:
# problems with default summaries - not specific enough.
# Solution - need to modify the prompt and pass it to Kura's existing summarise_conversations function

In [9]:
# Custom summary prompt (domain-specific for support cases)
# Ref: https://usekura.xyz/notebooks/how-to-look-at-data/02_summaries_task/#defining-our-new-summary-model
# Pass to analyze_conversations(..., prompt=CASE_SUMMARY_PROMPT). Use None for Kura default.
# To extend the default: from kura.summarisation import DEFAULT_SUMMARY_PROMPT; prompt=DEFAULT_SUMMARY_PROMPT + " Your extra instructions."

CASE_SUMMARY_PROMPT = """
Analyze the support case conversation to provide a focused summary for clustering.

In your response:

1. Identify the product area and case type, such as:
   - Payroll (submission, deadlines, earning codes, direct deposit)
   - Tax (notices, rate changes, SUIs, account maintenance)
   - HR & payroll (transfers, employee setup, supervisor visibility, time & labor)
   - Billing (invoices, fees, discrepancies)

2. State the user's concrete need in 20–30 words: what they asked for and how it was resolved (or not).

Format:
"User needed [product area]: [specific request or issue]. [Resolution type: e.g. guided to self-resolve, escalated, how-to.]"

Reference the conversation below:
<messages>
{% for message in conversation.messages %}
<message>{{message.role}}: {{message.content}}</message>
{% endfor %}
</messages>

You must also return these structured fields (they are captured on each summary):

- **summary**: (Use the format above; concise, no PII.)
- **request**: What is the user's overall request? Start with "The user's overall request for the assistant is to"
- **topic**: Optional short topic label (e.g. "payroll", "tax", "hr").
- **user_frustration**: Integer 1–5 (1=happy, 5=extremely frustrated).
"""

In [ ]:
# from kura.summarisation import SummaryModel
# from rich import print as rprint

# summaries = await summarise_conversations(
#    conversations=conversations[:2],
#    prompt=CASE_SUMMARY_PROMPT,
#    model=SummaryModel()
# )

# for summary in summaries:
#     rprint(summary)

Summarising 2 conversations: 100%|██████████| 2/2 [00:01<00:00,  1.16it/s]


ConversationSummary(
    summary='User needed HR & payroll: assistance with the holiday list for time & labor. Resolution type: guided 
to self-resolve.',
    request="The user's overall request for the assistant is to help with the holiday list in time & labor.",
    topic='hr',
    languages=None,
    task=None,
    concerning_score=None,
    user_frustration=1,
    assistant_errors=None,
    chat_id='65047504',
    metadata={'conversation_turns': 1, 'case_id': 65047504.0},
    embedding=None
)

ConversationSummary(
    summary='User needed HR & payroll: guidance on why their supervisor could not see all direct reports. 
Resolution type: guided to self-resolve by reviewing employee configuration and supervisor setup errors.',
    request="The user's overall request for the assistant is to provide guidance on supervisor visibility of direct
reports.",
    topic='hr',
    languages=None,
    task=None,
    concerning_score=None,
    user_frustration=1,
    assistant_errors=None,
    chat_id='65047964',
    metadata={'conversation_turns': 1, 'case_id': 65047964.0},
    embedding=None
)

In [11]:
async def analyze_conversations(conversations, checkpoint_manager):
    # Set up models
    summary_model = SummaryModel()
    cluster_model = ClusterDescriptionModel()
    meta_cluster_model = MetaClusterModel(max_clusters=4)
    dimensionality_model = HDBUMAP()

    # Run pipeline steps
    summaries = await summarise_conversations(
        conversations, model=summary_model, checkpoint_manager=checkpoint_manager, prompt=CASE_SUMMARY_PROMPT
    )

    clusters = await generate_base_clusters_from_conversation_summaries(
        summaries, model=cluster_model, checkpoint_manager=checkpoint_manager
    )

    reduced_clusters = await reduce_clusters_from_base_clusters(
        clusters, model=meta_cluster_model, checkpoint_manager=checkpoint_manager
    )

    projected = await reduce_dimensionality_from_clusters(
        reduced_clusters,
        model=dimensionality_model,
        checkpoint_manager=checkpoint_manager,
    )

    return projected

In [12]:
# run the pipeline
checkpoint_manager = JSONLCheckpointManager(CHECKPOINT_DIRECTORY, enabled=True)
checkpoint_manager.save_checkpoint("conversations", conversations)
clusters = await analyze_conversations(
    conversations, checkpoint_manager=checkpoint_manager
)

Summarising 48 conversations: 100%|██████████| 48/48 [00:04<00:00, 11.43it/s]


In [13]:
# Get top-level clusters (those without parents)
parent_clusters = [cluster for cluster in clusters if cluster.parent_id is None]

# Format each cluster's info with name, description and number of chats
formatted_clusters = []
for cluster in parent_clusters:
    cluster_info = (
        f"[bold]{cluster.name}[/bold] : {cluster.description} : {len(cluster.chat_ids)}"
    )
    formatted_clusters.append(cluster_info)

# Join with newlines and print
print("\n\n".join(formatted_clusters))

Resolve billing and check payment issues : The user addressed concerns regarding billing discrepancies and needed 
assistance with stop payments for checks. The resolutions guided the user towards self-help solutions for these 
financial matters. : 2

Clarify HR and payroll submission processes : Users sought guidance on payroll and HR submission procedures, as 
well as system issues related to these processes. They received instructions and information to help them resolve 
queries and system access concerns on their own. : 33

Help with tax rate changes and clarifications : Users requested assistance regarding various tax issues, focusing 
on rate changes and clarifications related to notices from tax agencies. They sought either guidance for 
self-resolution or support for escalating their cases for further investigation as necessary. : 13

In [14]:
# Analysing Our Summaries
from kura.types import ConversationSummary
from kura.checkpoints import JSONLCheckpointManager

checkpoint_manager = JSONLCheckpointManager(CHECKPOINT_DIRECTORY, enabled=True)
summaries = checkpoint_manager.load_checkpoint("summaries", ConversationSummary)
conversations = checkpoint_manager.load_checkpoint("conversations", Conversation)


id_to_conversation = {
    conversation.chat_id: conversation for conversation in conversations
}


for i in range(10):
    print(summaries[i].summary)
    print(id_to_conversation[summaries[i].chat_id].messages[0].content)

User needed HR & payroll: assistance with the holiday list for time & labor. Resolution type: guided to 
self-resolve.

product: time & labor | case type: support requested - time & labor | case detail: time & labor | subject: holiday 
| description: helped the client with the holiday list

User needed HR & payroll: guidance on why their supervisor could not see all direct reports. Resolution type: 
guided to self-resolve by reviewing employee configuration and supervisor setup errors.

product: hr & payroll | case type: 00 - how to/self resolved | case detail: how to/self resolved | subject: time & 
labor groups | description: client needed guidance as to why their supervisor could not see all of their direct 
reports. guided them to review the employee configuration and notices errors in supervisor setup

User needed HR & payroll: clarification on submitting payroll on 1/5/23 for a 1/6/23 check date. Resolution type: 
guided to self-resolve by providing submission options and advising on project communication.

...

product: tax account maintenance | case type: tax rate change | case detail: sui rate change | subject: florida 
2023 tax rate | description: 

hello!

please see xxxxx florida 2023 tax rate. thank you!


thank you,

User needed HR & payroll: clarification on payroll policies. Resolution type: guided to self-resolve.

product: time & labor | case type: support requested - time & labor | case detail: time & labor | subject: payroll 
policies | description: payroll policies

User needed HR & payroll: guidance on handling a system outage. Resolution type: self-resolved by waiting 15 
minutes.

product: hr & payroll | case type: 00 - how to/self resolved | case detail: how to/self resolved | subject: system 
outage | description: adv er to wait 15 min

In [15]:
from kura.visualization import visualise_pipeline_results

# Minimal tree
visualise_pipeline_results(clusters, style="basic")

# Default: tree + percentages + progress bars + stats
#visualise_pipeline_results(clusters, style="enhanced")

# Colored + tables (needs: pip install rich)
#visualise_pipeline_results(clusters, style="rich")

Clusters (48 conversations)
╠══ Resolve billing and check payment issues (2 conversations)
║   ╠══ Clarify billing discrepancies for December invoices (1 conversations)
║   ╚══ Assist with issuing stop payments for checks (1 conversations)
╠══ Clarify HR and payroll submission processes (33 conversations)
║   ╠══ Clarify payroll and HR submission processes (14 conversations)
║   ╚══ Assist with HR and payroll system issues (19 conversations)
╚══ Help with tax rate changes and clarifications (13 conversations)
    ╚══ Assist with tax rate changes and clarifications (13 conversations)



In [ ]:
# 